In [37]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.externals import joblib
pd.set_option('display.max_columns', None)  

In [38]:
df = pd.read_csv('../sample_csv_data/ohport_downsampling_iot_flow.csv')
df = df.drop(columns=['Unnamed: 0'])

In [39]:
df.head()

,timeStampFirst,timeStampLast,duration,linkProtocol,networkProtocol,transportProtocol,applicationProtocol,srcMac,dstMac,srcIP,dstIP,srcPort,dstPort,packetCount,totalPayloadByte,appplicationPayloadByte,iot,device,dns,dhcp-server,http,ntp,https,smtp-ssl,imap-ssl,ssdp,icslap,stm-pproc,stun,ws-discovery,upnp-evnt,xmpp,android,dstPort25050,dstPort49152,dstPort49153,dstPort49154,wellKnownDstPort,ephemeralDstPort,registedDstPort,srcPort49152,srcPort49153
0,1475671755714,1475671756741,1027,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,49153.0,3538.0,6,597,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1475020226003,1475020226003,0,Ethernet,IPv4,UDP,DNS,00:16:6c:ab:6b:88,14:cc:20:51:33:ea,192.168.1.249,192.168.1.1,50591.0,53.0,2,148,0,True,Samsung SmartCam,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1476208177398,1476208177427,29,Ethernet,IPv4,TCP,NaN,ec:1a:59:79:f4:89,ec:1a:59:83:28:11,192.168.1.223,192.168.1.193,49152.0,4018.0,7,4087,0,True,Belkin Wemo switch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1474599250064,1474599250086,22,Ethernet,IPv4,TCP,NaN,00:16:6c:ab:6b:88,ec:1a:59:83:28:11,192.168.1.249,192.168.1.193,49152.0,4415.0,5,1508,0,True,Samsung SmartCam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1476049394301,1476049394604,303,Ethernet,IPv4,TCP,NaN,44:65:0d:56:cc:d3,14:cc:20:51:33:ea,192.168.1.240,93.184.216.34,53974.0,80.0,4,272,0,True,Amazon Echo,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [40]:
# one hot for network & transport protocol
one_hot = pd.get_dummies(df['networkProtocol'])
df = df.join(one_hot)
one_hot = pd.get_dummies(df['transportProtocol'])
df = df.join(one_hot)

In [41]:
df.head()

,timeStampFirst,timeStampLast,duration,linkProtocol,networkProtocol,transportProtocol,applicationProtocol,srcMac,dstMac,srcIP,dstIP,srcPort,dstPort,packetCount,totalPayloadByte,appplicationPayloadByte,iot,device,dns,dhcp-server,http,ntp,https,smtp-ssl,imap-ssl,ssdp,icslap,stm-pproc,stun,ws-discovery,upnp-evnt,xmpp,android,dstPort25050,dstPort49152,dstPort49153,dstPort49154,wellKnownDstPort,ephemeralDstPort,registedDstPort,srcPort49152,srcPort49153,IPv4,IPv6,TCP,UDP
0,1475671755714,1475671756741,1027,Ethernet,IPv4,TCP,NaN,ec:1a:59:83:28:11,ec:1a:59:79:f4:89,192.168.1.193,192.168.1.223,49153.0,3538.0,6,597,0,True,Belkin wemo motion sensor,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,0,1,0
1,1475020226003,1475020226003,0,Ethernet,IPv4,UDP,DNS,00:16:6c:ab:6b:88,14:cc:20:51:33:ea,192.168.1.249,192.168.1.1,50591.0,53.0,2,148,0,True,Samsung SmartCam,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,1
2,1476208177398,1476208177427,29,Ethernet,IPv4,TCP,NaN,ec:1a:59:79:f4:89,ec:1a:59:83:28:11,192.168.1.223,192.168.1.193,49152.0,4018.0,7,4087,0,True,Belkin Wemo switch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0,1,0
3,1474599250064,1474599250086,22,Ethernet,IPv4,TCP,NaN,00:16:6c:ab:6b:88,ec:1a:59:83:28:11,192.168.1.249,192.168.1.193,49152.0,4415.0,5,1508,0,True,Samsung SmartCam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0,1,0
4,1476049394301,1476049394604,303,Ethernet,IPv4,TCP,NaN,44:65:0d:56:cc:d3,14:cc:20:51:33:ea,192.168.1.240,93.184.216.34,53974.0,80.0,4,272,0,True,Amazon Echo,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,1,0


In [42]:
features = [
    'dns',
    'dhcp-server',
    'http',
    'ntp',
    'https',
    'smtp-ssl',
    'imap-ssl',
    'ssdp',
    'icslap',
    'stm-pproc',
    'stun',
    'ws-discovery',
    'upnp-evnt',
    'xmpp',
    'android',
    'dstPort25050',
    'dstPort49152',
    'dstPort49153',
    'srcPort49152',
    'srcPort49153',
    'wellKnownDstPort',
    'ephemeralDstPort', 
    'registedDstPort',
    'IPv4',
    'IPv6',
    'TCP',
    'UDP',
    'duration',
    'packetCount',
    'totalPayloadByte',
    'appplicationPayloadByte'
]
X = df[features].values[:,:]
Y = df[['iot']].values[:]
Y = Y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

In [43]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("accurency is", accuracy_score(y_test,y_pred), "\nf1 score is", f1_score(y_test, y_pred), "\n")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


accurency is 0.9880260561667162 
f1 score is 0.9879962064760872 



In [44]:
joblib.dump(clf, 'saved_model.pkl')

['saved_model.pkl']

In [45]:
print(X_test[0])

[  1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   1.   1.   0.   0.   1.   0.
   2. 150.   0.]
